In [19]:
# Try using Google Cloud to store NLCD
from google.cloud import storage
import requests
import os
from tqdm import tqdm
from io import BytesIO

In [2]:
# Personal credentials
service_account = "sentinel2@uhi-causal-model.iam.gserviceaccount.com"
json_key = "../uhi-causal-model-a7e891b460a5.json"

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key

In [20]:
# Set up Google Cloud Storage client
client = storage.Client()
bucket = client.get_bucket('earth-engine_boston')

In [22]:
# Download file from the web and track the progress using tqdm
url = 'https://s3-us-west-2.amazonaws.com/mrlc/nlcd_2021_land_cover_l48_20230630.zip'
response = requests.get(url, stream=True)

total_size_in_bytes = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 KB

# Store the content in BytesIO
content = BytesIO()

# Use tqdm for the download progress bar
with tqdm(total=total_size_in_bytes, unit='B', unit_scale=True, desc='Downloading') as download_progress:
    for data in response.iter_content(block_size):
        content.write(data)
        download_progress.update(len(data))

download_progress.close()


Downloading: 100%|██████████| 1.96G/1.96G [06:30<00:00, 5.01MB/s]
Uploading:   0%|          | 0.00/1.96G [00:00<?, ?B/s]


TypeError: Blob.upload_from_file() got an unexpected keyword argument 'callback'

In [27]:
# Reset the content buffer to the beginning
content.seek(0)

# Upload content to Google Cloud Storage
blob = bucket.blob('nlcd2021.zip')

# Use tqdm for the upload progress bar
with tqdm(total=total_size_in_bytes, unit='B', unit_scale=True, desc='Uploading') as upload_progress:
    while True:
        chunk = content.read(1024)
        if chunk:
            blob.upload_from_file(file_obj=BytesIO(chunk), rewind=True)
            upload_progress.update(len(chunk))
        else:
            break
        

# Ensure the progress bars display completion
upload_progress.close()

Uploading:   0%|          | 13.3k/1.96G [00:04<167:17:51, 3.25kB/s]


TooManyRequests: 429 PUT https://storage.googleapis.com/upload/storage/v1/b/earth-engine_boston/o?uploadType=resumable&upload_id=ABPtcPpO4ftvyYsnX4DY9TPsIp5z7GJwUfXuGZfqj119u-HAnRW-4i6rKubSKdSky2oE0YzFAgdTBLiM7JJTg6aCfuudzD6yVckgELhnjfGXkIQ3uQ: {
  "error": {
    "code": 429,
    "message": "The object earth-engine_boston/nlcd2021.zip exceeded the rate limit for object mutation operations (create, update, and delete). Please reduce your request rate. See https://cloud.google.com/storage/docs/gcs429.",
    "errors": [
      {
        "message": "The object earth-engine_boston/nlcd2021.zip exceeded the rate limit for object mutation operations (create, update, and delete). Please reduce your request rate. See https://cloud.google.com/storage/docs/gcs429.",
        "domain": "usageLimits",
        "reason": "rateLimitExceeded"
      }
    ]
  }
}
: ('Request failed with status code', 429, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PERMANENT_REDIRECT: 308>)